# MODEL PREDICTING YEAR 2023 and 2024 GVA for every Crop

In [1]:
import warnings
from glob import glob
import glob
import os
import numpy as np
import plotly.express as px
from IPython.display import VimeoVideo
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from ipywidgets import interactive
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
def wrangle(filepath):
    df = pd.read_csv(filepath)

    # Replace specific values with 0
    df.fillna(0, inplace=True)
    df.replace('_', 0, inplace=True)
    df.replace('-', 0, inplace=True)
   
    return df


data_pattern = r"C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_*.csv"

# Use glob to find the file paths
files = glob.glob(data_pattern)

# Create a list to store DataFrames
frames = []

# Iterate over the files and apply the wrangle function
for file in files:
    df = wrangle(file)
    print(f"Processing {file}, df_shape: {df.shape}")
    frames.append(df)

Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_1.csv, df_shape: (162, 7)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_2.csv, df_shape: (180, 4)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_3.csv, df_shape: (90, 3)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_4.csv, df_shape: (98, 3)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_5.csv, df_shape: (1677, 4)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_6.csv, df_shape: (90, 3)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_7.csv, df_shape: (689, 4)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_8.csv, df_shape: (324, 4)
Processing C:\Users\fabiola\Desktop\TODAY\DATASET\data\Table_9.csv, df_shape: (210, 4)


In [3]:
df4=frames[3]

In [4]:
df4.head()
df=df4
df.columns

Index(['crop', 'years',
       'Main crops Gross Value Added in current prices (Value RWF per ha) '],
      dtype='object')

In [5]:
#pip install scikit-learn  


In [6]:
# Assuming you have a 'crop' column in your DataFrame
crop_column = 'crop'

# Selecting relevant columns for the model
features = ['years', crop_column]
target = 'Main crops Gross Value Added in current prices (Value RWF per ha) '

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Define a pipeline with preprocessing and modeling steps
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', 'passthrough', ['years']),  # Numeric column
            ('cat', OneHotEncoder(handle_unknown='ignore'), [crop_column])  # Categorical column (crop)
        ],
        remainder='drop'
    )),
    ('regressor', LinearRegression())
])

# Train the model using the pipeline
pipeline.fit(X_train, y_train)

# Print the model details
model = pipeline.named_steps['regressor']
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

# Define sliders and dropdown
years_slider = IntSlider(value=2023, min=2023, max=2024, step=1, description='Year')
crop_dropdown = Dropdown(options=X_train[crop_column].unique(), value=X_train[crop_column].unique()[0], description='Crop')

# Define the interactive function
def predict_gva(year, crop):
    data_to_predict = pd.DataFrame({'years': [year], 'crop': [crop]})
    predicted_gva = pipeline.predict(data_to_predict)
    print(f'Predicted GVA for {crop} in {year}: {predicted_gva[0]}')

# Create the interactive widget
interactive_plot = interactive(predict_gva, year=years_slider, crop=crop_dropdown)

# Display the interactive widget
interactive_plot


Intercept: -203142519.29706916
Coefficients: [ 101054.19384712  -90334.53456149 -474416.60201162  248204.03960894
 1559042.08273413  199842.97868093 -644877.93774833  659520.72733043
 -561101.62484028  755487.92561923  298363.21094792 -555101.29669213
 -830820.61704675   25388.16793985 -589196.51996082]


interactive(children=(IntSlider(value=2023, description='Year', max=2024, min=2023), Dropdown(description='Cro…